In [1]:
%load_ext autoreload
%autoreload 2

# <center> Ghub (Vhub) Exercise 2 </center>

## Overview

- Exercise to download a SRTM 30m Global 1 arc second V0003 DEM for a selected volcano and convert the DEM to GRASS GIS raster format.
- Demonstrates installing a Jupyter Book Github project on Ghub.
- Demonstrates running a Ghub Pegasus workflow with python scripts on CCR.

## Background

- The python scripts are encapsulated as a workflow. The Pegasus Workflow Management System (WMS) automates and manages the execution of the workflow jobs, including staging the jobs, distributing the work, submitting the jobs to run in parallel on CCR, as well as handling data flow dependencies and overcoming job failures. See https://theghub.org/tools/pegtut for an introductory Pegasus tutorial.
- This Jupyter-based tool uses Python 3. See https://theghub.org/resources?alias=jupyterexamples for more information on developing Jupyter-based tools on Ghub.
- This tool is deployed on Debian 10 to run in Tool or App mode style. See https://theghub.org/kb/development/deploy-styles-for-jupyter-tools for more information on deploying Jupyter-based tools on Ghub.



In [2]:
# TODO: Rename widgets to PEP 8 standard

# Setup and preoprocessing:

import sys
import os
import getpass
import platform
import shutil
import atexit
#import shlex
#import subprocess
import math
import numpy as np
import pandas as pd
import time
import datetime

import ipywidgets as widgets
from IPython.display import display, HTML, Markdown, clear_output, Image, Javascript
#import xml.etree.ElementTree as et

#from sklearn.neighbors import DistanceMetric

#print ('sys.path: ', sys.path)
#print ("os.environ['LD_LIBRARY_PATH']: ", os.environ['LD_LIBRARY_PATH'])

#print ('shutil.which("python"): ', shutil.which("python")) #/apps/share64/debian7/anaconda/anaconda-6/bin/python

# Setup paths to executables
scriptpath = os.path.realpath(" ")
        
# Get the parent dirs
self_tooldir = os.path.dirname(scriptpath)

# Setup path to python and bash scripts
self_bindir = os.path.join(self_tooldir, "bin")

# Add to PYTHONPATH
sys.path.insert (1, self_bindir)
#sys.path.insert(0, self_bindir)

# Setup path to python and bash scripts and puffin executables
self_libdir = os.path.join(self_tooldir, "lib")

# Add to PYTHONPATH
sys.path.insert (2, self_libdir)

print ('sys.path: ', sys.path)

import hublib
#print (help(hublib))
import hublib.ui as ui
#print (help(ui))
#import hublib.use
#print (help(hublib.use))

# Set up path to the current data directory
self_datadir = os.path.join(self_tooldir, "data")

# Set up path to the current session directory
self_workingdir = os.getcwd()

# Set up path to the user's home directory
self_homedir = os.path.expanduser("~")

# Initialize the dated run directory.
# Workflow results are not available until after a workflow is executed via Pegasus and completes
self_rundir = ""

self_user = getpass.getuser()

# Configuration parameters

import Configuration as cfg
if cfg.VERBOSE == True:
    print ('cfg.DISPERSION_MODEL: ', cfg.DISPERSION_MODEL, '\n')
#import GeoLocation

#from Utils import Coordinates
from Utils import Tree
from Utils import GeoLocation
from newthreading import Thread
from Wrapper import Wrapper

self_log_filepath = os.path.join(self_workingdir, 'ghub_vhub_exercise2_log_file.txt')
self_log_snapshot_filepath = os.path.join(self_workingdir, 'ghub_vhub_exercise2_log_snapshot_file.txt')
self_log_backup_filepath = os.path.join(self_workingdir, 'ghub_vhub_exercise2_log_backup_file.txt')

#np.set_printoptions(threshold=np.inf)    

if cfg.VERBOSE == True:
    
    print ("Operating System Platform: " + platform.system(), platform.release())
    print ("\n")

    print ("Environment:\n")
    print ("scriptpath: " + scriptpath)
    print ("tooldir: " + self_tooldir)
    print ("bindir: " + self_bindir)
    print ("datadir: " + self_datadir)
    print ("workingdir: " + self_workingdir)
    print ("homedir: " + self_homedir)
    print ("user: " + self_user)
    print (" ")
    
    #print ("puffin_exe_filepath: ", puffin_exe_filepath)
    #print (" ")
    
    print ("sys.path :", sys.path)
    print (" ")
    
    print ("os.environ['PATH']: ", os.environ["PATH"])
    print (" ")

widget_border_style = '1px solid black'
widget_output_border_style = '1px solid black'

BOLD = '\033[1m'
END = '\033[0m'

dropdown_str_width = 16

dropdown_width = '965px'
dropdown_height = '30px'
button_width = '250px'
button_height = '40px'
ui_string_width = '96.5%'
ui_dropdown_width = '96.2%'
ui_text_width = '50.0%'

# Clean up: remove files from the data/results folder and the bin/__pycache__ folder
def exit_handler():
    
    for file in os.listdir(self_workingdir):
        
        if os.path.isfile(file):
            if file.endswith(".txt"):
                if file != "README.txt":
                    print ("Deleting: %s\n" %file)
                    os.remove(file)
            #elif file.endswith(".dax"):
                #print ("Deleting: %s\n" %file)
                #os.remove(file)
            #elif file.endswith(".stdout"):
                #print ("Deleting: %s\n" %file)
                #os.remove(file)
            #elif file.endswith(".stderr"):
                #print ("Deleting: %s\n" %file)
                #os.remove(file)

    #dirpath = os.path.join(self_bindir, "__pycache__")
    #if (os.path.exists(dirpath)):
        #print ("Deleting: %s\n" %dirpath)
        #shutil.rmtree(dirpath)

atexit.register(exit_handler);   

sys.path:  ['/home/theghub/renettej/AAA_notebooks/ghub_vhub_exercise2', '/home/theghub/renettej/AAA_notebooks/ghub_vhub_exercise2/bin', '/home/theghub/renettej/AAA_notebooks/ghub_vhub_exercise2/lib', '/apps/share64/debian10/anaconda/anaconda-7/lib/python38.zip', '/apps/share64/debian10/anaconda/anaconda-7/lib/python3.8', '/apps/share64/debian10/anaconda/anaconda-7/lib/python3.8/lib-dynload', '', '/home/theghub/renettej/.local/lib/python3.8/site-packages', '/apps/share64/debian10/anaconda/anaconda-7/lib/python3.8/site-packages', '/apps/share64/debian10/anaconda/anaconda-7/lib/python3.8/site-packages/IPython/extensions', '/home/theghub/renettej/.ipython']


<IPython.core.display.Javascript object>

In [3]:
# prevent In[] and Out[] from displaying on left
#HTML('''
#<style>.prompt{width: 0px; min-width: 0px; visibility: collapse}</style>
#''')

In [4]:
#https://api.jquery.com/ready/
HTML('''
<script>
    function scroll_to_top() {
        Jupyter.notebook.scroll_to_top();
    } 
    $( window ).on( "load", scroll_to_top() );
</script>
''')

In [5]:
# Button styles
HTML('''
<style>.buttontextclass { color:black ; font-size:130%}</style>
''')

<a name="top"></a>
#### [**Processing Steps**]<br />

1. [Volcano Selection](#step_1) <br />
2. [Grass GIS Database Names Selection](#step_2) <br />
3. [Workflow](#step_3)<br />
    1. [Download SRTM 30m Global 1 arc second V0003 DEM for the selected volcano.](#step_3)<br />
    2. [Crate a GRASS GIS database and translate the DEM to GRASS GIS raster format.](#step_3)<br />
    3. [Transfer the created GRASS GIS database to Ghub.](#step_3)<br />
4. [View Workflow Output](#step_4)<br />
5. [View Log Output](#step_5)<br />



In [6]:
# #219F hex = #8607 decimal
# This works also works for an up arrow: [$\tiny\uparrow$](#top)

In [7]:
if os.path.exists(self_log_filepath):
    shutil.copy (self_log_filepath, self_log_backup_filepath)
    
FH1 = open(self_log_filepath, 'w')

show_log_output_button = widgets.Button(description="Show Log Output", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'lightgreen','font_weight':'bold'})

# Utility Function

def log_info (message):
    if show_log_output_button.description == 'Hide Log Output': 
        with log_output:
            print (message)    
    FH1.write('%s\n' %message)
    FH1.flush()
        
log_info ('self_user: ' + self_user)
log_info ('self_homedir: ' + self_homedir)
log_info ('self_tooldir: ' + self_tooldir)
log_info ('self_workingdir: ' + self_workingdir)
#log_info ('self_ismip6aissvn_models_dirpath: ' + self_ismip6aissvn_models_dirpath)
#log_info ('self_ismip6aissvn_scalars_dirpath: ' + self_ismip6aissvn_scalars_dirpath)

<a name="step_1"></a>
## Step 1:  Volcano Selection [&#8607;](#top)

Enter the first character of the volcano name to expedite the search. The "Volcano Grid Parameters" will be selected by the model, once the volcano is chosen.  Change the "Volcano Grid Parameters" only if you want to customize output. 

In [8]:
def get_database_volcano_name(volcano_name):
    database_volcano_name = volcano_name
    if "'S " in database_volcano_name:
        database_volcano_name =  database_volcano_name.replace("'S ", 'S_')
    elif ', ' in database_volcano_name:
        database_volcano_name =  database_volcano_name.replace(', ', '_')
    elif ' ' in database_volcano_name:
        database_volcano_name =  database_volcano_name.replace(' ', '_')
    return database_volcano_name
        
# TODO: Fix dropdown widget margin
volcanos = []

filepath = os.path.join(self_datadir, "volcanos.txt")

if (cfg.VERBOSE == True):
    print ('volcanos.txt filepath: ', filepath)
    
FH2 = open(filepath, "r")
for volcano in FH2:
    # Remove white space
    volcano.strip()
    values = volcano.split(":")
    if (values[0] != "Puff Volcano Listing File - do not remove this header\n"):
        values = [value.strip(" ") for value in values]
        volcanos.append(values)
FH2.close()

#print (len(volcanos))
#print (volcanos[1:5])

volcano_names = [row[0] for row in volcanos]
#print (type(volcanoNames))
#print (len(volcanoNames))
#print (volcanoNames)
#max_len_volcano_name = max(volcano_names, key = len)
#print (max_len_volcano_name) # CAMPI FLEGREI MAR SICILIA
#print (len(max_len_volcano_name)) # 25

# Default volcano
volcano_name = 'AZUFRAL, VOLCAN'
database_volcano_name = get_database_volcano_name(volcano_name)

#layout={'max-height': '100px', 'width': 'initial', 'justify_content': 'center', 'overflow-y': 'auto'})

volcano_dropdown = widgets.Dropdown(
    name='Volcano Name',
    description='Volcano Name',
    value=volcano_name,
    options=volcano_names,
    style = {'description_width': '90px'},
    layout={'height': '30px', 'width': '350px'})
    
#help(volcanoDropDown)

# Process information for the selected volcano

process_volcano_info_output = widgets.Output()

def process_volcano_info():
    
    global volcano_name
    global volcano_name_previous
    global volcano_lat
    global volcano_lat_hemisphere
    global volcano_lat_decimal_degrees
    global volcano_lon
    global volcano_lon_hemisphere
    global volcano_lon_decimal_degrees
    global volcano_height
    global current_datetime
    global self_default_radiosonde_filepath

    with process_volcano_info_output:
        
        clear_output()
    
        volcano_name = volcano_dropdown.value
        #print (volcano_name)
        #print (type(volcano_name))
        volcano_index = volcano_names.index(volcano_name)
        #print (volcanoIndex)
        
        #print (volcanos[volcano_index])
        volcano_location = volcanos[volcano_index][1]
        volcano_lat = float(volcanos[volcano_index][2])
        volcano_lat_hemisphere = volcanos[volcano_index][3]
        volcano_lon = float(volcanos[volcano_index][4])
        volcano_lon_hemisphere = volcanos[volcano_index][5]

        print ("%s, %s" %(volcano_name, volcano_location))
        
        # Decimal degrees
        sign = 1.0
        if (volcano_lat_hemisphere == 'S'):
            sign = -1.0
            #south = True
        #else:
            #south = False
        volcano_lat_decimal_degrees = sign * volcano_lat
        print ('\nLatitude:', volcano_lat_decimal_degrees, '[degrees north -90 to 90]')
        sign = 1.0
        if (volcano_lon_hemisphere == 'W'):
            sign = -1.0
        volcano_lon_decimal_degrees = sign * volcano_lon
        print ('Longitude:', volcano_lon_decimal_degrees, '[degrees east -180 to 180]')
        
        '''
        convert = Coordinates.Convert()
        transformed_point = convert.convert_from_decimal_degrees_to_utm(
            volcano_lat_decimal_degrees,
            volcano_lon_decimal_degrees
        )
        # Check @ https://www.latlong.net/lat-long-utm.html
        #print (transformed_point)
        print ('Latitude:', np.round(transformed_point[1], 2), '[UTMN]')
        print ('Longitude:', np.round(transformed_point[0], 2), '[UMTE]')
        '''
        
        # Get Lat/Lon Bounding box.
        # At the equator, 1 degree = 111 km
        #halfSideInKm = 555
        # Expects lat in -90 to 90, lon in -180 to 180
        self_geo_location = GeoLocation(0.0, 0.0, 0.0, 0.0)
        geo_loc = self_geo_location.from_degrees (volcano_lat_decimal_degrees, volcano_lon_decimal_degrees)
        #print ('geo_loc: ', geo_loc)
      
        #halfSideInKm = 111
        #111./8. = 13.875 [km] = 8.622 [mi]
        halfSideInKm = 13.875
        SW_loc, NE_loc = geo_loc.bounding_locations(halfSideInKm)
        #print (type(SW_loc), SW_loc)
        #print (type(NE_loc), NE_loc)
        
        # For the hysplit model, srm_2_nc.py gets an error when the lat/lon range values are not rounded. 
        # The benthysplitwf tool should also get modified for this.
        lat_south.value = round(SW_loc.deg_lat,2)
        lat_north.value = round(NE_loc.deg_lat,2)
        lon_west.value = round(SW_loc.deg_lon,2)
        lon_east.value = round(NE_loc.deg_lon,2)
        
        #print (float(gridLatRangeStart.value))
        #print (float(gridLatRangeStop.value))
        #print (utils.convert_lon_minus180_180_to_0_360(float(gridLonRangeStart.value)))
        #print (utils.convert_lon_minus180_180_to_0_360(float(gridLonRangeStop.value)))
       
        volcano_height = float(volcanos[volcano_index][6].rstrip())
        print ('Height: ', volcano_height, ' [m]\n')
        
        # Update grass_gis_database_form values
        database_volcano_name = get_database_volcano_name(volcano_name)
        grassgis_location.value = database_volcano_name + '_location'
        #grassgis_mapset.value = database_volcano_name + '_mapset'
        grassgis_map.value = database_volcano_name + '_map'
        
def volcano_name_change(change):
    
    if change['type'] == 'change' and change['name'] == 'value' and change['new'] != ' ':
        process_volcano_info()

volcano_dropdown.observe(volcano_name_change)

volcano_selection_form = ui.Form([volcano_dropdown], name = 'Volcano Selection')

lat_south = ui.Number(
    name = 'Latitude South',
    description = 'Latitude South [degrees north -90 to 90]',
    units = '',
    value = '0.0',
    min = '-90.0',
    max = '90.0'
)
lat_north = ui.Number(
    name = 'Latitude North',
    description = 'Latitude North [degrees north -90 to 90]',
    units = '',
    value = '0.0',
    min = '-90.0',
    max = '90.0'
)
lon_west = ui.Number(
    name = 'Longitude West',
    description = 'Longitude West [degrees east -180 to 180]',
    units = '',
    value = '0.0',
    min = '-180.0',
    max = '180.0'
)
lon_east = ui.Number(
    name = 'Longitude East',
    description = 'Longitude East [degrees east -180 to 180]',
    units = '',
    value = '0.0',
    min = '-180.0',
    max = '180.0'
)
volcano_coordinates_form = ui.Form([lat_south,
             lat_north,
             lon_west,
             lon_east], name = 'Volcano Bounding Box Coordinates')

volcano_parameters_form = \
    ui.Form([volcano_selection_form, process_volcano_info_output, volcano_coordinates_form], \
    name = 'Volcano Parameters')


In [9]:
display (volcano_parameters_form)

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Volcano Para…

## <a name="step_2"></a>
## Step 2: Grass GIS Database Name Selections[&#8607;](#top)
A GRASS GIS Database is a set of directories and files with certain structure which GRASS GIS works efficiently with. Location is a directory with data related to one geographic location or a project. All data within one Location has the same cartographic projection. A Location contains Mapsets and each Mapset contains data related to a specific task, user or a smaller project. Within each Location, a mandatory PERMANENT Mapset exists which can contain commonly used data within a Location such as base maps. PERMANENT Mapset also contains metadata related to Location such as projection. When GRASS GIS is started it connects to a Database, Location and Mapset specified by the user. <br/><br />See https://grass.osgeo.org/grass82/manuals/grass_database.html for more information.

In [10]:
HBox_layout = widgets.Layout(height='40px', width='98%', display='flex', flex_flow='row', justify_content='flex-start')
grassgis_database_label = widgets.Label(
    value = 'Database',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_database = widgets.Text(
    name='Database',
    value='grassdata',
    indent = False,
    layout={'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_database_box = widgets.HBox(children=[grassgis_database_label, grassgis_database], layout = HBox_layout)
grassgis_location_label = widgets.Label(
    value = 'Location',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_location = widgets.Text(
    name='Location',
    value=database_volcano_name + '_location',
    indent = False,
    layout={'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_location_box = widgets.HBox(children=[grassgis_location_label, grassgis_location], layout = HBox_layout)
# Getting errors when Mapset is not PERMANENT?
grassgis_mapset_label = widgets.Label(
    value = 'Mapset',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_mapset = widgets.Text(
    name='Mapset',
    value='PERMANENT',
    #value = database_volcano_name + '_mapset',
    indent = False,
    layout={'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_mapset_box = widgets.HBox(children=[grassgis_mapset_label, grassgis_mapset], layout = HBox_layout)
grassgis_map_label = widgets.Label(
    value = 'Map',
    layout = widgets.Layout(width='1000px',height='30px')
)
grassgis_map = widgets.Text(
    name='Map',
    value=database_volcano_name + '_map',
    indent = False,
    layout={'height': '30px', 'width': '500px'},
    style = {'description_width':'80px'},
    disabled = True
)
grassgis_map_box = widgets.HBox(children=[grassgis_map_label, grassgis_map], layout = HBox_layout)
grassgis_database_form = ui.Form(
    [grassgis_database_box, grassgis_location_box, grassgis_mapset_box, grassgis_map_box], name = 'Grass GIS Database')


In [11]:
display(grassgis_database_form)

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Grass GIS Da…

In [12]:
 # Run Workflow

maxwalltime = ui.Number(
    name = 'Maximum Walltime',
    description = 'Maximum Walltime [min]',
    units = 'min',
    value = '120.0',
    min = '60.0',
    max = '3600.0'
)

workflow_run_options_form = ui.Form([maxwalltime], name = 'Workflow Run Options')

def run_workflow(p):
    
    # print (p) #Button    
    
    workflow_output.clear_output()
        
    with workflow_output:
        
        print ('Workflow in progress...\n')
    
        runWorkflowButton.disabled = True
        
        start_time = time.time()

        try:
            
            self_workflow_results_filepath = os.path.join(self_workingdir, grassgis_database.value)
            log_info ('self_workflow_results_filepath: '+ self_workflow_results_filepath)
            
            if os.path.exists(self_workflow_results_filepath):
                log_info ('removing: ' + self_workflow_results_filepath)
                shutil.rmtree(self_workflow_results_filepath)
            
            python_launch_exec_path = os.path.join(self_tooldir, 'remotebin', 'pythonLaunch.sh')
            log_info('pythonLaunch.sh path: ' + python_launch_exec_path)

            #'''
            log_info ('About to call Wrapper...')
            self_tW = Thread(target=Wrapper, args=(" ", \
                self_tooldir, self_bindir, self_datadir, self_workingdir, self_rundir, \
                lat_south.value, lat_north.value, lon_west.value, lon_east.value,\
                grassgis_database.value, grassgis_location.value, grassgis_mapset.value, grassgis_map.value, int(maxwalltime.value)))

            # For Process:
            #self.t = Process(target=Wrapper,
            #args=(self, self.queue, ...))
            self_tW.daemon = True
            self_tW.start()

            # Wait for thread to complete
            self_tW.join()
            #'''

            # Check if utm.txt and deg.txt exist were created and tranferred form CCR 
            # to determine if workflow completed successfully

            if os.path.exists(self_workflow_results_filepath):

                print ("Workflow completed successfully\n")
                
                print ('\nDirectory tree of the GRASS GIS database:', self_workflow_results_filepath, '\n')
                results_tree = Tree(self_workflow_results_filepath)
                results_tree.print_tree()
                   
            else:

                print ("Workflow did not complete successfully")
                print ("%s not generated by the workflow\n" %self_workflow_results_filepath)

                filepath = os.path.join(self_workingdir, 'pegasus.analysis')
                if (os.path.exists(filepath)):
                    print("pegasus.analysis:\n")
                    FH1 = open(filepath, 'r')
                    output = FH1.read()
                    FH1.close()
                    print (output)

                print ("\nPlease see the log output\n")
        
        except Exception as e:
        
            print ("Workflow Exception: %s\n" %str(e))
            print ("Please see the log output\n")
       
        runWorkflowButton.disabled = False
            
        print ("\nWorkflow elapsed time: " + str((time.time() - start_time)/60.0) + " minutes\n")

        finish_workflow_processing()

# Abort
# Select Kernel Interrupt
#if self_tW.is_alive() == True:
   #self_tW.terminate()

runWorkflowButton = widgets.Button(description="Run Workflow", disabled=False,\
    layout=widgets.Layout(width=button_width, height=button_height),\
    style= {'button_color':'lightgreen','font_weight':'bold'})
runWorkflowButton.add_class("buttontextclass")
runWorkflowButton.on_click (run_workflow)
#help (runWorkflowButton)

# Note: See /apps/share64/debian7/anaconda/anaconda-6/lib/python3.7/site-packages/hublib/ui/pathselect.py,
# file property initialized to None, when a file is selected gets set to the selected file.


<a name="step_3"></a>
## Step 3: Run Workflow [&#8607;](#top)

Initiates the workflow.


In [13]:
display(workflow_run_options_form)
display(runWorkflowButton)

Group(children=(HTML(value="<p   style='background-color: #DCDCDC; font-size: 150%; padding: 5px'>Workflow Run…

Button(description='Run Workflow', layout=Layout(height='40px', width='250px'), style=ButtonStyle(button_color…

<a name="step_4"></a>
## Step 4: View Workflow Output [&#8607;](#top)


In [14]:
workflow_output = widgets.Output(layout={'border': '1px solid black'})
display(workflow_output)

Output(layout=Layout(border='1px solid black'))

In [15]:
def finish_workflow_processing():
    
    try:

        log_info ('\nfinish_workflow_processing...')
        
        # ghub_vhub_exercise2-workflow.dax is created by Wrapper.py
        #filepath = os.path.join(self_workingdir, 'ghub_vhub_exercise2-workflow.dax')
        #if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            #os.remove(filepath)

        for file in os.listdir(self_workingdir):
            if os.path.isfile(file):
                if file.startswith('python-') and file.endswith('.stdout'):
                    f = open(file,'r')
                    for line in f:
                        log_info (line)
                    f.close()
                    os.remove(file)
                    
        for file in os.listdir(self_workingdir):
            if os.path.isfile(file):
                if file.startswith('python-') and file.endswith('.stderr'):
                    f = open(file,'r')
                    for line in f:
                        log_info (line)
                    f.close()
                    os.remove(file)
         
        filepath = os.path.join(self_workingdir, 'pegasus.analysis')
        if (os.path.exists(filepath)):
            filesize = os.path.getsize(filepath)
            log_info ('pegasus.analysis filesize: ' + str(filesize))
            log_info ("pegasus.analysis:\n")
            f = open(filepath, 'r')
            output = f.read()
            f.close()
            log_info (output)
            os.remove(filepath)
        
        filepath = os.path.join(self_workingdir, 'pegasusstatus.txt')
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        filepath = os.path.join(self_workingdir, 'pegasusjobstats.csv')
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        filepath = os.path.join(self_workingdir, 'pegasussummary-time.csv')
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        filepath = os.path.join(self_workingdir, 'pegasussummary.csv')
        if os.path.exists(filepath):
            #print ("Deleting: %s\n" %filepath)
            os.remove(filepath)

        # send email to user
        '''
        email_subject = 'ghub exercise1 tool notification'
        if self_workflow_success:
            email_text = 'Create figures workflow completed successfully.'
        else:
            email_text = 'Create figures workflow completed with errors. See the View the Log Output File section for more information.'
        email_cmd = 'submit --progress silent mail2self -t "'+email_text+'" -s "'+email_subject+'"'
    
        # email debugging
        log_status (create_figures_button_callback_output, 'Sending completion email. Please wait approximately 1 minute...')
        log_info ('Sending completion email cmd: ' +email_cmd+'...')
    
        # Submit blocks
        start_time = time.time()
        os.system(email_cmd)
        elapsed_time = np.round((time.time() - start_time)/60.0, 2)
        log_status (create_figures_button_callback_output, 'Done. Elapsed time: ' + str(elapsed_time) + ' [min]')
        ''' 
        
        log_info ('finish_workflow_processing done.')
        
    except Exception as e:
        log_error (create_figures_button_callback_output, "EXCEPTION: %s\n" % str(e))


<a name="step_5"></a>
## Step 5: View Log Output [&#8607;](#top)

- If an error is encountered while running this tool,
the cause of the error will be written to the log output file, ghub_vhub_exercise2_log_file.txt.

- Click the `Show Log Output` button to open the `Log Output` window and view the log output file.


In [16]:
def show_log_output(change):
    
    if os.path.exists(self_log_filepath):
            
        if show_log_output_button.description == 'Show Log Output':
        
            show_log_output_button.description = 'Hide Log Output'
        
            with log_output:
            
                if os.path.exists(self_log_filepath):
                    print("%s: \n\n" %self_log_filepath)
                    f = open(self_log_filepath,'r')
                    for line in f:
                        print(line.rstrip())
                    f.close()
                else:
                    job_error (log_output, '%s does not exist ' %filepath + '. Please contact us.')
        else:
        
            show_log_output_button.description = 'Show Log Output'
            log_output.clear_output()
    else:
        job_error (log_output, '%s does not exist ' %filepath + '. Please contact us.')

show_log_output_button.add_class("buttontextclass")
show_log_output_button.on_click(show_log_output)
display (show_log_output_button)

Button(description='Show Log Output', layout=Layout(height='40px', width='250px'), style=ButtonStyle(button_co…

In [17]:
log_output = widgets.Output(layout={'border': widget_output_border_style})
display (log_output)

Output(layout=Layout(border='1px solid black'))

In [18]:
# Initialize widgets with default values

# Process default volcano
process_volcano_info()
